# 전체 로직
 #### 1. 검색하고 싶은 키워드(해시태그)를 리스트에 넣는다
 #### 2. 해당 키워드에 대한 페이지로 들어가게된다
 #### 3. 스크롤을 내리면서 각 게시물의 URL 크롤링
 #### 4. 수집한 URL을 CSV파일로 만듬
 #### 5. 리스트에 있는 모든 키워드에 대해 1~4 반복
 
 ##### cf) URL 크롤러와 TAG 크롤러를 구분했다 
 #####  => 수집된 URL에 대한  TAG 크롤링은 약 5~6개의 크롤러로 나눠서 돌리는게 훨씬 빠르다

In [3]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import re
import os
import csv
import time
import codecs

In [4]:
class UrlCrawller:
    last = "" # 이전에 크롤링된 URL 리스트의 마지막 URL -> 중복을 방지하기 위함
    count = 0 # 현재까지 크롤링된 URL 개수
    zero_repeat = 0
    
    #크롤러 객체 초기화
    def __init__(self, chromedriver_path, keyword, url_csv_filepath, target_num, SCROLL_PAUSE_TIME = 3):
        self.keyword = keyword
        self.target_url = "https://www.instagram.com/explore/tags/"+keyword
        self.driver = webdriver.Chrome(chromedriver_path)
        self.url_csv_filepath = url_csv_filepath
        self.target_num = target_num
        self.SCROLL_PAUSE_TIME = SCROLL_PAUSE_TIME
        UrlCrawller.last = ""
        UrlCrawller.count=0
        UrlCrawller.zero_repeat = 0
        
    #크롤러 객체 준비
    def crawlling_ready(self):
        #기존에 파일 있는지 유무 확인
        if(not os.path.exists(self.url_csv_filepath)):
            print('파일이 없으므로 생성 후 크롤링을 시작합니다')
        
        else:
            print('파일이 있으므로 기존의 파일에 덧붙이겠습니다.')
        
        self.driver.implicitly_wait(3) 
        self.driver.get(self.target_url)
        
        #포스팅 개수 추출, 문자열
        posting_gatsu_str = self.driver.find_element_by_xpath('.//span[@class="g47SY "]').text 
        
        #문자열로 추출된 개수를 정수형으로 변환
        posting_gatsu = int(posting_gatsu_str.replace(",", ""))
        
        #사용자가 원하는 크롤링 개수보다 전체 게시물이 적을 경우
        if self.target_num > posting_gatsu:
            self.target_num = posting_gatsu
        
        print("--------------태그 : %s -------------------" %self.keyword)
        print("총 게시물 : ", posting_gatsu)
        self.url_crawlling()
    
    
    # 새롭게 크롤링 된 URL 리스트에 이전에 수집된 URL 리스트 목록에 있는 일부 요소들이 중복되어 들어가있다. 
    # 따라서 URL을 긁어올 떄 마다 해당 리스트의 마지막 URL을 저장해놓고 
    # 새롭게 크롤링 된 URL 리스트에서 이전 리스트의 마지막 URL 다음부터 저장할 수 있도록 한다
    def url_crawlling(self):
        UrlCrawller.last = ""
        while True:
            source = self.driver.page_source # 바이트코드 type으로 소스를 읽는다.
            soup = BeautifulSoup(source, 'lxml') # 파싱할 문서를 BeautifulSoup 클래스의 생성자에 넘긴다
            item = soup.findAll('a', {"href": re.compile('^/p/')}) # 해당 영역 html 형식 -> <a href="/p/Bk_Jqc5gNYC/?tagged=sydney">
            item_len = len(item)
            position = 0 # 이전 리스트와 중복되지 않는 부분부터 시작하기 위한 변수
            new_item = [] # 이전 리스트와 중복되지 않는 부분부터 시작되는 새로운 리스트
            
            # 두 번째 크롤링부터는 이전 리스트의 마지막 url 값이 공백이 아니므로 이를 처리하기 위함
            if UrlCrawller.last!="":
                for i in item:
                    if UrlCrawller.last!=i.attrs['href']:
                        position+=1 
                        continue
                    
                    else:
                        position+=1
                        break
            
            # 이전 리스트와 중복되지 않는 부분부터 시작되는 새로운 리스트를 만든다
            for i in range(position,item_len):
                current_href = item[i].attrs['href'] # 해당 영역 html 형식 -> <a href="/p/Bk_Jqc5gNYC/?tagged=sydney">
                new_item.append(current_href)
                
                if i==item_len-1:
                    UrlCrawller.last = item[i].attrs['href']
           
            #새로운 리스트 길이
            new_item_len = len(new_item)
              
            #새로운 리스트가 이전 리스트와 동일한 경우
            if new_item_len==0:
                UrlCrawller.zero_repeat+=1
                time.sleep(1)
                print("응답 없음 : ",UrlCrawller.zero_repeat)
                
                if UrlCrawller.zero_repeat==10:
                    # 새로 고침한뒤 다시 시작
                    # 마지막 url 있는데까지 스크롤 내리기
                    return self.refresh_start_over(UrlCrawller.count)
                
                continue
            
            else:
                UrlCrawller.zero_repeat = 0
                #url 을 저장하기 위한 csv 파일작성     
                self.write_csv_url(new_item)
                
            
            # url 크롤링 몇 개까지 완료되었는지
            UrlCrawller.count+= new_item_len
            print("(%d / %d)" %(UrlCrawller.count, self.target_num))
            
            if(UrlCrawller.count >= self.target_num):
                #return print("%s URL 크롤링 완료!\n\n" %self.keyword)
                print("%s URL 크롤링 완료!\n\n" %self.keyword)
                self.driver.close()
                break
             
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(self.SCROLL_PAUSE_TIME)
     
    
    # 새로 고침한뒤 다시 시작
    # 수집된 개수만큼 움직였을 떄 다시 수집
    def refresh_start_over(self,success_cnt):
        print("새로고침!")
        print("success_cnt : ",success_cnt)
        UrlCrawller.zero_repeat = 0
        self.driver.refresh()
        time.sleep(7)
        
        re_count = 0
        UrlCrawller.last=""
        
        while True:
            source = self.driver.page_source # 바이트코드 type으로 소스를 읽는다.
            soup = BeautifulSoup(source, 'lxml') # 파싱할 문서를 BeautifulSoup 클래스의 생성자에 넘긴다
            item = soup.findAll('a', {"href": re.compile('^/p/')}) # 해당 영역 html 형식 -> <a href="/p/Bk_Jqc5gNYC/?tagged=sydney">
            item_len = len(item)
            position = 0 # 이전 리스트와 중복되지 않는 부분부터 시작하기 위한 변수
            new_item = [] # 이전 리스트와 중복되지 않는 부분부터 시작되는 새로운 리스트
            
            if UrlCrawller.last!="":
                for i in item:
                    if UrlCrawller.last!=i.attrs['href']:
                        position+=1 
                        continue
                    
                    else:
                        position+=1
                        break
            
            # 이전 리스트와 중복되지 않는 부분부터 시작되는 새로운 리스트를 만든다
            for i in range(position,item_len):
                current_href = item[i].attrs['href'] # 해당 영역 html 형식 -> <a href="/p/Bk_Jqc5gNYC/?tagged=sydney">
                new_item.append(current_href)
                
                if i==item_len-1:
                    UrlCrawller.last = item[i].attrs['href']
           
            #새로운 리스트 길이
            new_item_len = len(new_item)
            
            #새로운 리스트가 이전 리스트와 동일한 경우
            if new_item_len==0:
                UrlCrawller.zero_repeat+=1
                time.sleep(1)
                
                if UrlCrawller.zero_repeat==10:
                    return self.refresh_start_over(success_cnt)
                
                continue
            
            else:
                UrlCrawller.zero_repeat = 0
                re_count+= new_item_len
                print("re count : ", re_count)
                if re_count >= success_cnt:
                    return self.url_crawlling()
                
            
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(self.SCROLL_PAUSE_TIME)


    #url 을 저장하기 위한 csv 파일을 만든다     
    def write_csv_url(self,list):
        with codecs.open(self.url_csv_filepath,"a","utf-8") as fp:
            writer=csv.writer(fp,delimiter=",")
            writer.writerow(list)

In [5]:
def main(keyword):
    url_csv_filepath = 'C:/Users/acorn/weather_bigdata/original_url/url_'+keyword+'.csv'
    chromedriver_path = 'C:/Users/acorn/Desktop/weather/chromedriver_win32/chromedriver'
    target_url = "https://www.instagram.com/explore/tags/"+keyword
    target_num = 100000 # 크롤링할 게시물 개수
    crawller = UrlCrawller(chromedriver_path, keyword, url_csv_filepath, target_num)
    crawller.crawlling_ready()

In [6]:
keyword = input("크롤링할 태그를 입력하세요 : ")
main(keyword)

크롤링할 태그를 입력하세요 : 제주도
파일이 있으므로 기존의 파일에 덧붙이겠습니다.
--------------태그 : 제주도 -------------------
총 게시물 :  8630535
(21 / 100000)
(45 / 100000)
(57 / 100000)


WebDriverException: Message: chrome not reachable
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 6.1.7601 SP1 x86_64)
